# Applying XGBoost to guess multiple points in the future

In this Jupyter Notebook, we use XGBoost to predict M points in the future from the N previous points. The input include all the features of the São Cristóvão station and the output include M points with one of the cumulative rain feature (that can be chagned manually)

## Preparation

In this section we prepare some tool, modules, and preprocessing on the dataset to run the experiment.

In [ ]:
import os
import sys
import re
import pandas as pd
import numpy as np
import json

import subprocess

from matplotlib import pyplot as plt

from time import time
from threading import Thread
from threading import Lock

import sklearn as sk
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
import xgboost as xgb

In [ ]:
if "data/output" not in os.getcwd():
    os.chdir("data/output")

### Loading the dataset

In this section the dataset is loaded and unified as one pandas dataframe.

In [ ]:
data = {}
station = "sao_cristovao"
sources = ['AlertaRio_DadosPluv', 'AlertaRio_DadosMet']
data[station] = {}
source = sources[0]
data[station][source] = data[station][source] = pd.read_csv(source + "/full/" + station + ".csv", sep=',')
source = sources[1]
data[station][source] = data[station][source] = pd.read_csv(source + "/full/" + station + ".csv", sep=',')

for source in sources:
    data[station][source]['datetime'] = pd.to_datetime(data[station][source]['Dia'] + data[station][source]['Hora'], format='%d/%m/%Y%H:%M:%S')
    data[station][source].set_index('datetime', inplace=True)
    data[station][source] = data[station][source].asfreq("15T")["2000":"2023-05-18 02:00:00"]
data[station][sources[1]].drop(columns=["Chuva"], inplace=True)

Checking for the same range of date

In [ ]:
data["sao_cristovao"]["AlertaRio_DadosPluv"].iloc[0].name == \
data["sao_cristovao"]["AlertaRio_DadosMet"].iloc[0].name, \
\
data["sao_cristovao"]["AlertaRio_DadosPluv"].iloc[-1].name == \
data["sao_cristovao"]["AlertaRio_DadosMet"].iloc[-1].name

Concatenate the 2 datasets and remove useless columns

In [ ]:
drop_list = ['Dia', 'Hora']
data_features = pd.concat([data["sao_cristovao"]["AlertaRio_DadosPluv"].drop(columns=drop_list), data["sao_cristovao"]["AlertaRio_DadosMet"].drop(columns=drop_list)], axis=1)

### Features extraction

Extract the features from the date to run XGBoost.

In [ ]:
def get_date_features(df):
    out = df.copy()
    out["min"] = out.index.minute
    out["hour"] = out.index.hour
    out["day"] = out.index.day
    out["month"] = out.index.month
    out["year"] = out.index.year
    
    out['quarter'] = out.index.quarter
    out['dayofyear'] = out.index.dayofyear
    out['dayofmonth'] = out.index.day
    
    out['weekofyear'] = out.index.isocalendar().week.astype(np.int64)
    return out

Defining the number of points to predict in the future and choose the target (cumulative rain).

In [ ]:
N_target = 96  # Number of shifted columns
target = '04h'

targets = [target]

# Create shifted columns
for i in range(1, N_target + 1):
    targets.append(f'Fut_{target}{i}')

### Missing data

Dealing with missing data using linear regression and using only the data from 2002 to today as the station don't provide all the features in it's first 2 years of work.

In [ ]:
data_features['2002':][target].isna().sum()

In [ ]:
data_features[target] = data_features[target].interpolate(method='linear')

data_features = data_features['2010':]

data_features[target].isna().sum()

### Group points

In this section the data from the previous points and the target of the next M points is added.

In [ ]:
def get_colums_names(column_names, N):
    column_names = list(column_names)
    names = []
    for i in range(N, 0, -1):
        for name in column_names:
            names.append(name + str(i))
    names.extend(column_names)
    return names

data_features.reset_index(inplace=True)
all_available_features = list(data_features.columns)

N = 15 # Number of points to predict future
data_multiple = data_features.copy()

for i in range(1, N):
    data_multiple = pd.concat([data_multiple.iloc[:-1].reset_index(drop=True), data_features.iloc[i:].reset_index(drop=True)], axis=1)

data_multiple = pd.concat([data_multiple.iloc[:-1].reset_index(drop=True), data_features.iloc[N:].reset_index(drop=True)], axis=1)

data_multiple.columns = get_colums_names(data_features.columns, N)

data_multiple.drop(columns=["datetime" + str(i) for i in range(1, N + 1)], inplace=True)
data_multiple.set_index("datetime", inplace=True)

# Create shifted columns
for i in range(1, N_target + 1):
    col_name = targets[i]
    data_multiple[col_name] = data_multiple[target].shift(-i)

# data_multiple = get_date_features(data_multiple)
data_multiple = data_multiple[:-N_target]

Checking if there is missing data in the targets.

In [ ]:
data_multiple[targets].isna().sum()

### Partitionning

Partitionning the dataset as below:

4 partitions:

- Training Extreme Event
- Training Usual Weather
- Testing Extreme Event
- Testing Usual Weather

In [ ]:
# Used if you want to use MAPE as score metric function
# data_multiple += 1e-8

In [ ]:
t = pd.Timedelta("15T")

def date_to_index(date, starting_date):
    return (pd.to_datetime(date, format='%d/%m/%Y') - starting_date) / t

def index_to_date(index, starting_date):
    return starting_date + index * t

In [ ]:
extreme_events_dates = pd.to_datetime(pd.DataFrame(["19/03/2000",
                                                    "11/06/2006",
                                                    "04/04/2010",
                                                    "06/04/2010",
                                                    "25/04/2011",
                                                    "26/04/2011",
                                                    "12/03/2016",
                                                    "14/02/2018",
                                                    "15/02/2018",
                                                    "08/04/2019",
                                                    "09/04/2019"])[0], format='%d/%m/%Y')
extreme_events_index = [date_to_index(date, data_multiple.index.min()) for date in extreme_events_dates]

In [ ]:
extreme_events_index

Creating the Extreme events partition by splitting 4 extreme event in training and 2 in testing.

In [ ]:
extreme_event_list = [(605300, 605700),
                        (565240, 565440),
                        (497620, 497820),
                        (326480, 326700),
                        (289500, 289850),
                        (155710, 155900)]

extreme_event_list = [(int(i) - 150, int(i) + 150) for i in extreme_events_index if i > 0]

extreme_sep = 4

extreme_events = pd.DataFrame()
for i, j in extreme_event_list:
    extreme_events = pd.concat([extreme_events, data_multiple[i:j]])
    
extreme_training = pd.DataFrame()
for i, j in extreme_event_list[:extreme_sep]:
    extreme_training = pd.concat([extreme_training, data_multiple[i:j]])
    
extreme_testing = pd.DataFrame()
for i, j in extreme_event_list[extreme_sep:]:
    extreme_testing = pd.concat([extreme_testing, data_multiple[i:j]])


Creating usual weather partition (picking random segment and check that they aren't extreme event.

In [ ]:
np.random.uniform(3, 30, (2, 5 ))

nb_seg_training = 100
nb_seg_testing = 400
len_seg = 10

random_training = pd.DataFrame()

for _ in range(nb_seg_training): # Number of random segment
    while True:
        ran = np.random.randint(0, len(data_multiple))
        ok = True
        for i, j in extreme_event_list:
            if i in range(ran, ran + len_seg):
                ok = False
            if j in range(ran, ran + len_seg):
                ok = False
        if ok:
            break
    random_training = pd.concat([random_training, data_multiple[ran:ran + len_seg]])

random_testing = pd.DataFrame()

for _ in range(nb_seg_testing): # Number of random segment
    while True:
        ran = np.random.randint(0, len(data_multiple))
        ok = True
        for i, j in extreme_event_list:
            if i in range(ran, ran + len_seg):
                ok = False
            if j in range(ran, ran + len_seg):
                ok = False
        if ok:
            break
    random_testing = pd.concat([random_testing, data_multiple[ran:ran + len_seg]])

## Training and evaluating

Defining the training features.

In [ ]:
all_features = data_multiple.columns

training_features_list = ['15min', '01h', '04h', '24h', '96h', 'DirVento',
       'VelVento', 'Temperatura', 'Pressao', 'Umidade', 'hour', 'day', 'month',
       'year', 'quarter', 'dayofyear', 'dayofmonth', 'weekofyear']

# training_features_list = ['15min', '01h', '04h', '24h', '96h', 'DirVento',
#        'VelVento', 'Temperatura', 'Pressao', 'Umidade']

# Features to avoid for each point (If a feature like 15min is in this list,
# then training feature will contains 15min1, ..., 15minN but not 15min)
avoid_features = ['datetime', '15min', '01h', '04h', '24h', '96h', 'DirVento',
       'VelVento', 'Temperatura', 'Pressao', 'Umidade']

def is_training_feature(feature, training_features, avoid_features):
    for training_feature in training_features:
        if feature not in avoid_features and \
        training_feature == feature[:len(training_feature)] and \
        (feature[len(training_feature):].isnumeric() or feature[len(training_feature):] == ""):
            return True
    return False

training_features = list(filter(lambda x : is_training_feature(x, training_features_list, avoid_features), all_features))

### Training the model

In [ ]:
args = {
    "n_estimators" : 200,
    "learning_rate" : 0.05
}

reg_list = []

# This condition allow to use more than one metric
# If you don't have a Nvidia GPU remove the tree_method argument in the XGBoost regressor
if True:
    """One metric"""
    reg = xgb.XGBRegressor(tree_method="gpu_hist",
                           **args)
    reg_list.append(reg)
else:
    """Multiple metrics"""
    reg_both = xgb.XGBRegressor(tree_method="gpu_hist",
                           eval_metric=['rmse', 'mape'],
                           **args)
    reg_rmse = xgb.XGBRegressor(tree_method="gpu_hist",
                           eval_metric=['rmse'],
                           **args)
    reg_mape = xgb.XGBRegressor(tree_method="gpu_hist",
                           eval_metric=['mape'],
                           **args)
    reg_list.append(reg_both)
    reg_list.append(reg_rmse)
    reg_list.append(reg_mape)

for reg in reg_list:
    print("\nstarting with :", reg.eval_metric)
    tmp = time()
    
    if True:
        reg.fit(pd.concat([extreme_training[training_features], random_training[training_features]]), # Training partition = extreme event + usual weather
                pd.concat([extreme_training[targets], random_training[targets]]), # Training targets
                eval_set=[(extreme_training[training_features], extreme_training[targets]), # Evaluating set = all partition
                         (extreme_testing[training_features], extreme_testing[targets]),
                          (random_training[training_features], random_training[targets]),
                         (random_testing[training_features], random_testing[targets])],
                verbose=20)
    else:
        reg.set_params(learning_rate=1)
        
        print("Training with random data")
        reg.fit(random_training[training_features], random_training[targets],
                eval_set=[(random_training[training_features], random_training[targets]),
                         (random_testing[training_features], random_testing[targets])],
                verbose=50)

        reg.set_params(learning_rate=0.01)
        print("Training with Extreme event")
        reg.fit(extreme_training[training_features], extreme_training[targets],
                eval_set=[(extreme_training[training_features], extreme_training[targets]),
                         (extreme_testing[training_features], extreme_testing[targets])],
                verbose=50)
    print("Time :", time() - tmp)

In [ ]:
extreme_events_index # Usefull to manually ajust the index

Plotting using a manual starting index.

In [ ]:
start_index = 324880
    
print(start_index)
print(index_to_date(start_index, data_multiple.index.min())) # Print the date of the start index

for reg in reg_list:
    print("\nstarting with :", reg.eval_metric)

    preds_test = reg.predict(data_multiple[training_features][start_index:start_index + 1])
    
    fig = plt.figure(figsize=(15, 10))
    grid = plt.GridSpec(2, 2, figure=fig)

    # Plotting the first subplot data_features(top)
    ax1 = fig.add_subplot(grid[0, :])
    ax1.set_title('Testing Data/predicted value')
    ax1.plot(data_multiple[start_index:start_index + N_target + 1][target], color="blue")
    ax1.plot(data_multiple.index[start_index:start_index + N_target + 1], preds_test[0], alpha=0.7, color="red")
    ax1.legend(['Testing Set', 'Prediction'])

    # Plotting the second subplot (bottom left)
    ax2 = fig.add_subplot(grid[1, 0], sharey=ax1)
    ax2.set_title('Testing Data')
    ax2.plot(data_multiple[start_index:start_index + N_target + 1][target], color="blue")
    ax2.legend(['Testing Set'])

    # Plotting the third subplot (bottom right)
    ax3 = fig.add_subplot(grid[1, 1], sharey=ax1)
    ax3.set_title('Prediction value')
    ax3.plot(data_multiple.index[start_index:start_index + N_target + 1], preds_test[0], color="red")
    ax3.legend(['Prediction'])

    plt.tight_layout()

    plt.show()

Plotting using a random starting index.

In [ ]:
start_index = np.random.randint(0, len(data_multiple))
    
print(start_index)
print(index_to_date(start_index, data_multiple.index.min())) # Print the date of the start index

for reg in reg_list:
    print("\nstarting with :", reg.eval_metric)

    preds_test = reg.predict(data_multiple[training_features][start_index:start_index + 1])

    fig = plt.figure(figsize=(15, 10))
    grid = plt.GridSpec(2, 2, figure=fig)

    # Plotting the first subplot (top)
    ax1 = fig.add_subplot(grid[0, :])
    ax1.set_title('Testing Data/predicted value')
    ax1.plot(data_multiple[start_index:start_index + N_target + 1][target], color='blue')
    ax1.plot(data_multiple.index[start_index:start_index + N_target + 1], preds_test[0], alpha=0.7, color='red')
    ax1.legend(['Testing Set', 'Prediction'])

    # Plotting the second subplot (bottom left)
    ax2 = fig.add_subplot(grid[1, 0], sharey=ax1)
    ax2.set_title('Testing Data')
    ax2.plot(data_multiple[start_index:start_index + N_target + 1][target], color='blue')
    ax2.legend(['Testing Set'])

    # Plotting the third subplot (bottom right)
    ax3 = fig.add_subplot(grid[1, 1], sharey=ax1)
    ax3.set_title('Prediction value')
    ax3.plot(data_multiple.index[start_index:start_index + N_target + 1], preds_test[0], color="red")
    ax3.legend(['Prediction'])

    plt.tight_layout()

    plt.show()

## Conclusion

The model is extremely inconsistant, sometimes the prediction is very good, sometimes the model predict an extreme event when there is no rain, sometime it predict no rain when there is an intense rainfall. And if the model predict an extreme event, 2 points later it can predict nothing as it is highly sensitive.

Those experiments show that more research is needed in this field and highlight the difficulty of the process of predicting extreme events.

The diffuclty of the experiment can be explain by those following points:

- Few amount of data on extreme event
- Extreme event characteristics may vary
- Not seasonal

However, there is hope in improving the results with the following aspect:

- Use other features:
    - Other meteorological station
    - Other data sources such as oceanic probe, radars, etc
- Spend more time on training tweaking :
    - Hyperparameters
    - Training partition
    - Cross-validation
- Use a better interpolation method on the missing data (A linear interpolation was made)